## Let's simulate multiple comparisons for this power analysis

### Two drugs (Drug A and Drug B) with a placebo control.
Each participant will take either placebo or one of the two drugs for 7 days per condition.
We want to detect a 1 point difference in responses to the drugs. Presume that the STDEV is 2.
Find out how many participants are needed per group, and compare anova, mulitple comparison correction (p vs. drug a, p vs. drug b, drug a vs. drug b). Simulate results from a repeated measures anova.
Adjust the parameters below to test cohort size. 

Assuming in the first two that you'll average within-individual measures for the inter-group comparison and that variance will be consistent. 

In [ ]:
#Note 45 people gets you to a 0.9 power with a KW simulation. More lenient; probably ok given many reps per person.

## Using stats model power anova to test

In [23]:
import statsmodels.stats.power as smp

# Set parameters
effect_size = 1/2  # # Desired effect size (difference of 2 points divided by standard dev of 3 points)
alpha = 0.05
power = 0.90  # Adjust as needed
#nobs=50
n_conditions = 3  # Placebo, Drug A, Drug B
n_measurements = 28  # Total measurements per participant (7 for placebo, 7 for drug)

# Calculate the required sample size for each group
nobs = smp.FTestAnovaPower().solve_power(effect_size=effect_size, nobs=None, alpha=alpha, power=power, k_groups=n_conditions)

# Calculate the total required sample size (participants) needed
total_sample_size = nobs * n_measurements

print(f"Effect Size: {int(effect_size)}")
print(f"Required sample size per group: {int(nobs)} participants")
print(f"Total sample size: {int(total_sample_size)} measures")


Effect Size: 0
Required sample size per group: 53 participants
Total sample size: 1504 measures


### With a correction for multiple comparisons - placebo vs. drug a, placebo vs. drug b, drug a vs. drug b

In [15]:
import statsmodels.stats.power as smp
import numpy as np

# Set parameters
effect_size = 2/3  # Desired effect size (difference of 1 point divided by standard dev of 2)
alpha = 0.05  # Individual significance level
power = 0.90  # Desired power
n_subjects_per_group = 30  # Number of subjects per group
n_conditions_per_group = 2  # Number of conditions per group
n_measurements_per_condition = 14  # Number of measurements per condition
n_comparisons = 3  # Number of pairwise comparisons (placebo + drug A), (placebo + drug B)

# Calculate adjusted alpha to control familywise error rate (e.g., Bonferroni correction)
adjusted_alpha = alpha / n_comparisons

# Calculate power analytically for adjusted alpha
nobs = smp.FTestAnovaPower().solve_power(effect_size, nobs=None, alpha=adjusted_alpha, power=power, k_groups=n_conditions_per_group)

# Print the required sample size for adjusted alpha
print(f"Required sample size for adjusted alpha ({adjusted_alpha}): {int(nobs)} subjects")


Required sample size for adjusted alpha (0.016666666666666666): 33 subjects


## Alternately, simulate repeated measures anova. This simulation takes a while to run, use with caution 

In [9]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

In [10]:
# Set parameters
effect_size = 0.5  # Desired effect size
alpha = 0.05  # Significance level
power = 0.90  # Desired power
n_measurements = 14*45*2 # Number of measurements per subject (7 days in a week, 2 weeks), times the number of participants, times the number of groups (placebo+a, placebo +b)
simulations = 10  # Number of simulations

# Initialize a counter for significant results
significant_results = 0

In [ ]:
# Perform simulations
for _ in range(simulations):
    # Generate synthetic data
    np.random.seed()  # Random seed for each simulation
    data = {
        'Subject': np.repeat(range(n_measurements), n_measurements),
        'Measurement': np.tile(range(n_measurements), n_measurements),
        'Value': np.random.normal(loc=effect_size, scale=1, size=n_measurements ** 2)
    }

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Fit a repeated measures ANOVA model
    model = smf.ols('Value ~ Measurement + C(Subject)', data=df).fit()

    # Perform ANOVA
    aov_table = sm.stats.anova_lm(model, typ=2)

    # Check if the p-value is less than alpha
    if aov_table['PR(>F)']['C(Subject)'] < alpha:
        significant_results += 1

In [27]:

# Calculate power as the proportion of significant results
calculated_power = significant_results / simulations

# Print the estimated power
print(f"Estimated Power: {calculated_power:.3f}")


Estimated Power: 0.100
